In [200]:
import random
import math

## The init method initializes all parameters to zero. 

### If any parameter needs to have a default value, you can set it here. 

### All penalty parameters are set to a default value of 10

In [201]:
class PrimerDesign(object):
    
    def __init__ (self, name):
        #parameters are based on the biology handout
        '''parameters for the length criterion'''
        self.max_length = 22
        self.min_length = 18
        self.penalty_length = 10
        
        '''parameters for the temperature difference criterion'''
        self.max_tdiff = 1
        self.min_tdiff = 0
        self.penalty_tdiff = 10
        
        '''parameters for the cg content criterion'''
        self.max_cg = 0.6
        self.min_cg = 0.4
        self.penalty_cg = 10
        
        '''parameters for the annealing temperature criterion'''
        self.max_temp = self.max_length*(0.6)*(4) + self.max_length*(0.4)*(2) #70.4 degrees celcius
        self.min_temp = self.min_length*(0.4)*(4) + self.min_length*(0.6)*(2) #50.4 degrees celcius
        self.penalty_temp = 10
        
        '''parameters for the run criterion'''
        self.run_threshold = 4
        self.penalty_runs = 10
        
        '''parameters for the repeat criterion'''
        self.repeat_threshold = 1
        self.penalty_repeats = 10
        
        '''parameters for the specificity criterion'''
        self.penalty_specificity = 10 
        
        '''locations where the forward primer should be chosen from'''
        self.fp_start = 301
        self.fp_end = 600
        
        '''locations where the reverse primer should be chosen from'''
        self.rp_start = 601
        self.rp_end = 901
        
        ''' parameters for the simulated annealing portion'''
        self.initial_temperature = 70.4 #this is self.max_temp aka max annealing temperature
        self.stopping_temperature = 0.01
        self.drop_fraction = 0.999
        
        

### Task 2 
This method takes in a string variable containing the raw gene sequence and cleans it up to leave only the alphabet characters a,t,c,g. The result is then stored in the instance variable dna_sequence to be used in other methods.

In [202]:
#removes all non-alphabet characters from the selected dna sequence
class PrimerDesign(PrimerDesign): 

    def set_dna_sequence(self, dna_sequence):
        if type(dna_sequence) != str:
            return "ERROR, dna sequence is not a string."
        elif dna_sequence == '':
            return 0
        #changes the string into a list containing all the elements, and recombines it into a string with all non-alphabet characters excluded
        else:
            dna_sequence = ''.join([i for i in dna_sequence if i.isalpha()]).strip("\n")
            self.dna_sequence = dna_sequence
            return dna_sequence
#Test
a = PrimerDesign("BIODW")
c = "attggg 235 ttgccgt" 
b = a.set_dna_sequence(c)
print(b)

attgggttgccgt


### Task 3
This method generates a forward primer or reverse primer at random with a specified length of 20bp long. 

In [203]:
class PrimerDesign(PrimerDesign):
    
    def func_select_random(self, sqtype='forward', length = 20 ):
        if self.dna_sequence == 0:
            return 'ERROR, dna sequence is not a string'
        #ensure length is positive
        if length < 0:
            return ('Length has to be a positive number!')
        
        '''the length has to be a positive number'''
        
        if(sqtype == 'forward'):
            start_limit = self.fp_start 
            end_limit = self.fp_end 
            #initiating starting position of the forward primer randomly
            startt = random.randint(start_limit, end_limit - length +1)
            #find the end position of the forward primer
            endd = startt + length
            #slice the dna sequence to obtain the forward primer with correct length
            fprimer = self.dna_sequence[startt:endd+1]
            return fprimer
            
        elif(sqtype == 'reverse'):
        
            start_limit = self.rp_start 
            end_limit = self.rp_end
            #initiating starting position of the reverse primer randomly
            startt = random.randint(start_limit, end_limit-length+1)
            #find the end position of the reverse primer
            endd = startt + length
            #slice the dna sequence to obtain the complement of the reverse primer with correct length
            rprimer = self.dna_sequence[startt:endd+1]
            #we do not complement it since it which will yield the same values for ALL cost function as the reverse primer,hence we will use the complement for the subsequent functions.
            return rprimer


### Task 4
The first three methods determine the length of the sequence, the fraction of 'c's and 'g's in the string of dna sequence, and the annealing temperature. Followed by a method that shows the number of runs throughout the dna sequence; a run is indicated by a base that consecutively appears for more than the run threshold, which is 4 times. The last method determines the number of repeats of two distict bases that appear consecutively. 

If the input is not a string, the functions will return an error message. If the input is an empty string, the functions will return a 0 value. 

There are several test cases from eDimension below the last method.



In [204]:
class PrimerDesign(PrimerDesign): 
            
    def func_length(self, sq):
        if type(sq) != str:
            return "ERROR, dna sequence is not a string."
        else:
            return len(sq)
    
    def func_cg_fraction(self, sq):
        if type(sq) != str:
            return "ERROR, dna sequence is not a string."
        elif sq =="":
            return 0
        else:
            dic = {'a' :0, 'c' : 0, 'g' : 0, 't' : 0}
            for alphabet in sq:
                if alphabet in dic:
                    dic[alphabet]+=1
            cgfrac = (dic["c"] + dic["g"])/sum(dic.values())
            return cgfrac
    
    def func_temperature(self,sq):
        if type(sq) != str:
            return "ERROR, dna sequence is not a string."
        else:
            dic = {'a' :0, 'c' : 0, 'g' : 0, 't' : 0}
            for alphabet in sq:
                if alphabet in dic:
                    dic[alphabet]+=1
            Tsq = 4*(dic['g']+dic['c']) + 2*(dic['a']+dic['t'])
            return Tsq
            

In [205]:

class PrimerDesign(PrimerDesign):

    def func_count_runs(self,sq):
        if type(sq) != str:
            return "ERROR, dna sequence is not a string."
        elif sq == '':
            return 0
        else:
            first = sq[0]
            #a is the count of the number of times a base appears consecutively
            a = 0
            lst = []
            for alphabet in sq:
                if alphabet == first:
                    #if the same base appears again, add on to the count "a"
                    a += 1
                else: 
                    #adds on to the list the number of consecutive appearance once base stops appearing consecutively
                    lst.append(a)
                    #restarts the count
                    a = 0
                    first = alphabet
                    a+=1
            lst.append(a)
            counter = 0
            for i in lst:
                #if base appears consecutively four times, count for runs increase by one
                if i > self.run_threshold:
                    counter+=1
            return counter
        

In [206]:

class PrimerDesign(PrimerDesign):
    def func_count_repeats(self,sq):
        if type(sq) != str:
            return "ERROR, dna sequence is not a string."
        
        elif sq == '':
            return 0
        
        else:
            di_repeats = ['at','ac','ag','ca','ct','cg','ga','gt','gc','ta','tc','tg']
            lst = list(sq)
            anss = 0
            for i in range(len(lst)-3):
                #if two or more of the same bases appear consecutively, it is NOT counted as a repeat
                if lst[i] == lst[i+1]:
                    pass
                #if two bases appear consecutively, add on to the count "anss"
                elif lst[i] == lst[i+2] and lst[i+1] == lst[i+3]:
                    anss+=1
            return anss

#TEST CASE 1   
a = PrimerDesign("BIODW")
b = 'cattaaaaatacgaaaaaagtcat'
print(a.func_length(b))
print(a.func_cg_fraction(b))
print(a.func_temperature(b))
print(a.func_count_runs(b))
print(a.func_count_repeats(b))

#TEST CASE 2 - this test ensures that func_cg_fraction runs properly (no "g"s and "c"s in dna sequence) and that runs and repeats are computed accurately
b = 'atatatatattttatatataa'
print(a.func_length(b))
print(a.func_cg_fraction(b))
print(a.func_temperature(b))
print(a.func_count_runs(b))
print(a.func_count_repeats(b))

#TEST CASE 3 - this test ensures that empty string returns 0 for all functions
b = ''
print(a.func_length(b))
print(a.func_cg_fraction(b))
print(a.func_temperature(b))
print(a.func_count_runs(b))
print(a.func_count_repeats(b))

#TEST CASE 4 - this test ensures that only strings should be accepted as input, otherwise returns an error message
b = 1233
print(a.func_length(b))
print(a.func_cg_fraction(b))
print(a.func_temperature(b))
print(a.func_count_runs(b))
print(a.func_count_repeats(b))

#TEST CASE 5 - this test made sure that real and long dna sequences can be computed by the methods as well
b = dna_sequence
print(a.func_length(b))
print(a.func_cg_fraction(b))
print(a.func_temperature(b))
print(a.func_count_runs(b))
print(a.func_count_repeats(b))


24
0.20833333333333334
58
2
0
21
0.0
42
0
12
0
0
0
0
0
ERROR, dna sequence is not a string.
ERROR, dna sequence is not a string.
ERROR, dna sequence is not a string.
ERROR, dna sequence is not a string.
ERROR, dna sequence is not a string.
11838
0.3966067864271457
27988
36
309


### Task 5
Each of these class methods calculates the cost of each previous criteria of the dna sequence, e.g. length, runs, repeats, etc. If the cost function for a particular criterion returns zero, the criterion is met. 

Several test cases from eDimension are displayed below.


In [207]:
class PrimerDesign(PrimerDesign):
    
    def cost_length(self, sq):
        if type(sq) != str:
            return "ERROR, dna sequence is not a string."
        
        elif sq == '':
            return 0
        
        else:
            '''This is given to you as an example '''
            sq_len = len(sq)
            if(sq_len > self.max_length):
                return (sq_len - self.max_length)*self.penalty_length
            elif(sq_len < self.min_length):
                return (self.min_length - sq_len)*self.penalty_length 
            else:
                return 0
    
    def cost_temperature(self,sq):
        if type(sq) != str:
            return "ERROR, dna sequence is not a string."
        
        elif sq == '':
            return 0
        
        else:
            Tsq = self.func_temperature(sq)
            if Tsq > self.max_temp:
                return self.penalty_temp*(Tsq - self.max_temp)
            elif self.min_temp <= Tsq <= self.max_temp:
                return 0
            elif Tsq < self.min_temp:
                return self.penalty_temp*(self.min_temp - Tsq)
        
    def cost_cgcontent(self,sq):
        if type(sq) != str:
            return "ERROR, dna sequence is not a string."
        
        elif sq == '':
            return 0
        
        else:
            c = self.penalty_cg
            cgfrac = self.func_cg_fraction(sq)

            if cgfrac > self.max_cg:
                return c*(cgfrac - self.max_cg)
            elif self.min_cg <= cgfrac <= self.max_cg:
                return 0
            elif cgfrac < self.min_cg:
                return c*(self.min_cg - cgfrac)
        
    def cost_temperature_difference(self, fp, rp):
        if type(fp) != str or type(rp) != str:
            return "ERROR, dna sequence is not a string."
        
        elif fp == '' or rp =="":
            return 0
        
        else:
            dic = {'a' :0, 'c' : 0, 'g' : 0, 't' : 0}
            dc = {'a' :0, 'c' : 0, 'g' : 0, 't' : 0}
            for alphabet in fp:
                if alphabet in dic:
                    dic[alphabet]+=1 
            for alphabet in rp:
                if alphabet in dc:
                    dc[alphabet]+=1 
            Tapr = 4*(dic['g']+dic['c']) + 2*(dic['a']+dic['t'])
            Tarp = 4*(dc['g']+dc['c']) + 2*(dc['a']+dc['t'])

            deltaT = abs(Tapr - Tarp) 
            if deltaT > self.max_tdiff:
                return self.penalty_tdiff*(deltaT-self.max_tdiff)
            else:
                return 0
    
    def cost_specificity(self, sq): 
        if type(sq) != str:
            return "ERROR, dna sequence is not a string."
        
        elif sq == '':
            return 0
        
        else:

            countdna = self.dna_sequence.count(sq)
            if countdna > 0:
                return (countdna-1)*self.penalty_specificity
            else:
                return 0
    
    
    def cost_runs(self, sq):
        if type(sq) != str:
            return "ERROR, dna sequence is not a string."
        
        elif sq == '':
            return 0
        
        else:
            b = self.penalty_runs
            count = self.func_count_runs(sq)
            return b*count
    
    def cost_repeats(self,sq):
        if type(sq) != str:
            return "ERROR, dna sequence is not a string."
        
        elif sq == '':
            return 0
        
        else:
            r = self.penalty_repeats
            if r>0:
                anss = self.func_count_repeats(sq)
            else:
                anss = 0

            return r*anss
        
#Test cases 1-3 are from eDimension to test all the cost functions are working properly

#TEST CASE 1
a = PrimerDesign("BIODW")
b = 'cattaaaaatacgaaaaaagtcat'
a.set_dna_sequence(b)
print(a.cost_length(b))
print(a.cost_temperature(b))
print(a.cost_cgcontent(b))
print(a.cost_runs(b))
print(a.cost_repeats(b))
print(a.cost_specificity(b))
print("\n")

#TEST CASE 2
b = 'atatatatattttatatataa'
a.set_dna_sequence(b)
print(a.cost_length(b))
print(a.cost_temperature(b))
print(a.cost_cgcontent(b))
print(a.cost_runs(b))
print(a.cost_repeats(b))
print(a.cost_specificity(b))
print("\n")

#TEST CASE 3 - ensures empty strings have no cost functions.
b = ''
a.set_dna_sequence(b)
print(a.cost_length(b))
print(a.cost_temperature(b))
print(a.cost_cgcontent(b))
print(a.cost_runs(b))
print(a.cost_repeats(b))
print(a.cost_specificity(b))
print("\n")

#TEST CASE 4 - this test ensures that only strings should be accepted as input, otherwise returns an error message.
b = 12345
a.set_dna_sequence(b)
print(a.cost_length(b))
print(a.cost_temperature(b))
print(a.cost_cgcontent(b))
print(a.cost_runs(b))
print(a.cost_repeats(b))
print(a.cost_specificity(b))
print("\n")

#TEST CASE 5 - this test ensures cost functions can run properly on real and long dna sequences.
b = dna_sequence
a.set_dna_sequence(b)
print(a.cost_length(b))
print(a.cost_temperature(b))
print(a.cost_cgcontent(b))
print(a.cost_runs(b))
print(a.cost_repeats(b))
print(a.cost_specificity(b))
print("\n")

#TEST CASE 6 - this test ensures that the cost function for cost_temperature_difference works
a = PrimerDesign("BIODW")
c = dna_sequence
b = a.set_dna_sequence(c)
fp = a.func_select_random(sqtype='forward', length =20)
rp = a.func_select_random(sqtype='reverse', length =20)
print(a.cost_temperature_difference(fp, rp))

20
0
1.9166666666666667
20
0
0


0
83.99999999999999
4.0
0
120
0


0
0
0
0
0
0


ERROR, dna sequence is not a string.
ERROR, dna sequence is not a string.
ERROR, dna sequence is not a string.
ERROR, dna sequence is not a string.
ERROR, dna sequence is not a string.
ERROR, dna sequence is not a string.


118160
279176.0
0.033932135728543256
360
3090
0


10


### Task 6
This is a method to determine the objective function, which is the sum of all the cost functions described in the previous section. It gives you a score that tells you how close these primers are to the optimal primer. A score of zero tells you that your primers selected have met all the criteria. When the score is close to zero, you know that the primers that you have selected is close to the optimal choice. 


In [208]:
class PrimerDesign(PrimerDesign):
    
    def cost_objective_function(self, fp, rp):
        '''complete the calculation of the cost'''
        cost = 0 
        cost+=self.cost_length(fp)
        cost+=self.cost_length(rp)
        cost+=self.cost_temperature(fp)
        cost+=self.cost_temperature(rp)
        cost+=self.cost_cgcontent(fp)
        cost+=self.cost_cgcontent(rp)
        cost+=self.cost_specificity(fp)
        cost+=self.cost_specificity(rp)
        cost+=self.cost_runs(fp)
        cost+=self.cost_runs(rp)
        cost+=self.cost_repeats(fp)
        cost+=self.cost_repeats(rp)
        cost+=self.cost_temperature_difference(fp,rp)
        
        return cost 

#Test 1-4 ensures that all the previous tests run smoothly and checks whether length of primers affect the cost objective function
#Test based on dna_sequence
a = PrimerDesign("BIODW")
c = dna_sequence
b = a.set_dna_sequence(c)
fp = a.func_select_random(sqtype='forward', length =18)
rp = a.func_select_random(sqtype='reverse', length =18)

print(a.cost_objective_function(fp,rp))
print("\n")

#Test based on dna_sequence(2)
a = PrimerDesign("BIODW")
c = dna_sequence
b = a.set_dna_sequence(c)
fp = a.func_select_random(sqtype='forward', length =19)
rp = a.func_select_random(sqtype='reverse', length =19)

print(a.cost_objective_function(fp,rp))
print("\n")

#Test based on dna_sequence(3)
a = PrimerDesign("BIODW")
c = dna_sequence
b = a.set_dna_sequence(c)
fp = a.func_select_random(sqtype='forward', length =20)
rp = a.func_select_random(sqtype='reverse', length =20)

print(a.cost_objective_function(fp,rp))
print("\n")

#Test based on dna_sequence(4)
a = PrimerDesign("BIODW")
c = dna_sequence
b = a.set_dna_sequence(c)
fp = a.func_select_random(sqtype='forward', length =21)
rp = a.func_select_random(sqtype='reverse', length =21)

print(a.cost_objective_function(fp,rp))
print("\n")

105.36842105263156


50


10.190476190476192


40




### Task 7
This instance method displays all the criteria and corresponding cost function score for each of the forward primer and reverse primer in a table. The table provides useful information to clearly see which criterion is met or not met for any pair of primers, e.g. if the cost function for a particular criterion returns zero, the criterion is met. 

In [209]:
class PrimerDesign(PrimerDesign):
    
    
    def cost_objective_function_info(self, fp, rp):
        txt = ""
        txt+= "===Forward Primer=== {}\n".format(fp)
        txt += "{0:<25} {1:>25}\n".format("Criterion","Cost function score")
        txt += "-"*51 + "\n"
        txt += "{0:<25} {1:>25.3f}\n".format("length",self.cost_length(fp))
        txt += "{0:<25} {1:>25.3f}\n".format("annealing temperature",self.cost_temperature(fp))
        txt += "{0:<25} {1:>25.3f}\n".format("%cg_content",self.cost_cgcontent(fp))
        txt += "{0:<25} {1:>25.3f}\n".format("specificity",self.cost_specificity(fp))
        txt += "{0:<25} {1:>25.3f}\n".format("runs",self.cost_runs(fp))
        txt += "{0:<25} {1:>25.3f}\n\n\n".format("repeats",self.cost_repeats(fp))
        #the following codes change the complement of reverse primer from method self.func_select_random(self, sqtype='reverse', length = 20 ) into the reverse primer
        #by replacing all the "c"s to "g"s and vice versa, and "a"s to "t"s and vice versa, then reversing the string to give the reverse primer in 5'- to -3' direction.
        a = rp.replace("c","g1")
        b = a.replace("g","c")
        c = b.replace('a','t1')
        d = c.replace('t','a')
        e = d.replace('a1', 't')
        f = e.replace('c1','g')
        rpc = f[::-1]
        txt+= "===Reverse Primer=== {}\n".format(rpc)
        txt += "{0:<25} {1:>25}\n".format("Criterion","Cost function score")
        txt += "-"*51 + "\n"
        txt += "{0:<25} {1:>25.3f}\n".format("length",self.cost_length(rp))
        txt += "{0:<25} {1:>25.3f}\n".format("annealing temperature",self.cost_temperature(rp))
        txt += "{0:<25} {1:>25.3f}\n".format("%cg_content",self.cost_cgcontent(rp))
        txt += "{0:<25} {1:>25.3f}\n".format("specificity",self.cost_specificity(rp))
        txt += "{0:<25} {1:>25.3f}\n".format("runs",self.cost_runs(rp))
        txt += "{0:<25} {1:>25.3f}\n".format("repeats",self.cost_repeats(rp))
        txt += "\n{0:<25} {1:>25.3f}\n".format("Total costs:",self.cost_objective_function(fp, rp))
        txt += "\n{0:<25} {1:>25.3f}\n".format("Temperature Difference",self.cost_temperature_difference(fp,rp))
        crit = ['length','annealing temperature','%cg_content','specificity','runs','repeats']
        data = [self.cost_length(fp), self.cost_temperature(fp), self.cost_cgcontent(fp)  ,self.cost_specificity(fp),self.cost_runs(fp),self.cost_repeats(fp)] 
        data2 = [self.cost_length(rp), self.cost_temperature(rp), self.cost_cgcontent(rp)  ,self.cost_specificity(rp),self.cost_runs(rp),self.cost_repeats(rp)] 
        fpfail = []
        rpfail = []
        for i in range(len(data)):
            if data[i] != 0 :
                fpfail.append(crit[i])
        for j in range(len(data)):
            if data2[j] != 0:
                rpfail.append(crit[j])
        fpnotmet = ''
        rpnotmet =''
        for i in range(len(fpfail)):
            fpnotmet += ("-" + fpfail[i] + "\n")
        for j in range(len(rpfail)):
            rpnotmet += ("-" + rpfail[j] + "\n")
            
        txt += "\n\nForward primer criteria is not met:\n{}\nReverse primer criteria is not met:\n{}".format(fpnotmet,rpnotmet)
        return txt

#Test based on dna_sequence
a = PrimerDesign("BIODW")
c = dna_sequence
b = a.set_dna_sequence(c)
fp = a.func_select_random(sqtype='forward', length =20)
rp = a.func_select_random(sqtype='reverse', length =20)     
print(a.cost_objective_function_info(fp,rp))

===Forward Primer=== atctcactaaaagaatagata
Criterion                       Cost function score
---------------------------------------------------
length                                        0.000
annealing temperature                         0.000
%cg_content                                   1.619
specificity                                   0.000
runs                                          0.000
repeats                                      10.000


===Reverse Primer=== attcgtgtagttgaaccctga
Criterion                       Cost function score
---------------------------------------------------
length                                        0.000
annealing temperature                         0.000
%cg_content                                   0.000
specificity                                   0.000
runs                                          0.000
repeats                                      10.000

Total costs:                                 91.619

Temperature Difference    

### Task 10
This method finds the forward primer and the reverse primer that will give an optimal cost objective function; all the criteria are met. This is done through generating possible solutions using random numbers to decide the primer pairs and updating the cost objective function associated such that it will find the primer pairs that corresponds to an optimal value of 0.

In [210]:
import random
class PrimerDesign(PrimerDesign): 
    
    def func_simulated_annealing(self):

        def get_primer_neighbour(sq):
            length = len(sq)
            oldstartposition = b.find(sq)
            newlength = random.randint(self.min_length, self.max_length + 1)
            if oldstartposition >= (self.fp_end - length):
                newstart = oldstartposition + random.randint(-4,-1)
                newend = newstart + newlength
                newfp = self.dna_sequence[newstart:newend+1]
            elif oldstartposition == (self.fp_start):
                newstart= oldstartposition + random.randint(1,4)
                newend = newstart + newlength
                newfp = self.dna_sequence[newstart:newend+1]
            else:
                newstart= oldstartposition + random.randint(-4,4)
                newend = newstart + newlength
                newfp = self.dna_sequence[newstart:newend+1]
            return newfp

        def get_primer_neighbour2(sq):
            length = len(sq)
            oldstartposition = b.find(sq)
            newlength = random.randint(self.min_length, self.max_length + 1)
            if oldstartposition >= (self.rp_end - length):
                newstart = oldstartposition + random.randint(-4,-1)
                newend = newstart + newlength
                newrp = self.dna_sequence[newstart:newend+1]
            elif oldstartposition == (self.rp_start):
                newstart= oldstartposition + random.randint(1,4)
                newend = newstart + newlength
                newrp = self.dna_sequence[newstart:newend+1]
            else:
                newstart= oldstartposition + random.randint(-4,4)
                newend = newstart + newlength
                newrp = self.dna_sequence[newstart:newend+1]
            return newrp


        temperature = self.initial_temperature
        stopping_temperature = self.stopping_temperature
        drop = self.drop_fraction      
        fp_current = a.func_select_random(sqtype='forward', length =20)
        rp_current = a.func_select_random(sqtype='reverse', length =20)
        cost = self.cost_objective_function(fp_current, rp_current)
        current_cost = cost

        while temperature>stopping_temperature:
            fp_new = get_primer_neighbour(fp_current)
            rp_new = get_primer_neighbour2(rp_current)
            new_cost = self.cost_objective_function(fp_new, rp_new)
            if( new_cost < current_cost ):
                fp_current = fp_new
                rp_current = rp_new
                current_cost = new_cost 
            else:
                delta = new_cost - current_cost
                acceptance_probability = math.exp(-delta/temperature) 
                num = random.uniform(0.0,1.0)
                if ( acceptance_probability > num):
                    fp_current = fp_new
                    rp_current = rp_new
                    current_cost = new_cost 
                else:
                    pass
            temperature*=drop
        #the method returns the optimal forward primer, reverse primer and the cost objective function
        return fp_current, rp_current, current_cost


### Store the DNA sequence given to you in the variable below 
We used allele 1 of Homo Sapiens APC gene regulator of WNT signaling pathway (from Biology worksheet)

In [211]:
dna_sequence = '''1 gtattggtgc agcccgccag ggtgtcactg gagacagaat ggaggtgctg ccggactcgg
61 aaatggggac acatgcattg gtggccaaaa gagagaggag acaaaaccgc tgcagatggc
121 tgatgtgaat ctagtggaaa gagctactgg ggatgagaga aagaggagga ggcaggaagt
181 acttaaacaa ctacaaggaa gtattgaaga tgaagctatg gcttcttctg gacagattga
241 tttattagag cgtcttaaag agcttaactt agatagcagt aatttccctg gagtaaaact
301 gcggtcaaaa atgtccctcc gttcttatgg aagccgggaa ggatctgtat caagccgttc
361 tggagagtgc agtcctgttc ctatgggttc atttccaaga agagggtttg taaatggaag
421 cagagaaagt actggatatt tagaagaact tgagaaagag aggtcattgc ttcttgctga
481 tcttgacaaa gaagaaaagg aaaaagactg gtattacgct caacttcaga atctcactaa
541 aagaatagat agtcttcctt taactgaaaa tttttcctta caaacagata tgaccagaag
601 gcaattggaa tatgaagcaa ggcaaatcag agttgcgatg gaagaacaac taggtacctg
661 ccaggatatg gaaaaacgag cacagcgaag aatagccaga attcagcaaa tcgaaaagga
721 catacttcgt atacgacagc ttttacagtc ccaagcaaca gaagcagaga ggtcatctca
781 gaacaagcat gaaaccggct cacatgatgc tgagcggcag aatgaaggtc aaggagtggg
841 agaaatcaac atggcaactt ctggtaatgg tcagggttca actacacgaa tggaccatga
901 aacagccagt gttttgagtt ctagtagcac acactctgca cctcgaaggc tgacaagtca
961 tctgggaacc aagatacgcg cttactgtga aacctgttgg gagtggcagg aagctcatga
1021 accaggcatg gaccaggaca aaaatccaat gccagctcct gttgaacatc agatctgtcc
1081 tgctgtgtgt gttctaatga aactttcatt tgatgaagag catagacatg caatgaatga
1141 actaggggga ctacaggcca ttgcagaatt attgcaagtg gactgtgaaa tgtatgggct
1201 tactaatgac cactacagta ttacactaag acgatatgct ggaatggctt tgacaaactt
1261 gacttttgga gatgtagcca acaaggctac gctatgctct atgaaaggct gcatgagagc
1321 acttgtggcc caactaaaat ctgaaagtga agacttacag caggttattg cgagtgtttt
1381 gaggaatttg tcttggcgag cagatgtaaa tagtaaaaag acgttgcgag aagttggaag
1441 tgtgaaagca ttgatggaat gtgctttaga agttaaaaag gaatcaaccc tcaaaagcgt
1501 attgagtgcc ttatggaatt tgtcagcaca ttgcactgag aataaagctg atatatgtgc
1561 tgtagatggt gcacttgcat ttttggttgg cactcttact taccggagcc agacaaacac
1621 tttagccatt attgaaagtg gaggtgggat attacggaat gtgtccagct tgatagctac
1681 aaatgaggac cacaggcaaa tcctaagaga gaacaactgt ctacaaactt tattacaaca
1741 cttaaaatct catagtttga caatagtcag taatgcatgt ggaactttgt ggaatctctc
1801 agcaagaaat cctaaagacc aggaagcatt atgggacatg ggggcagtta gcatgctcaa
1861 gaacctcatt cattcaaagc acaaaatgat tgctatggga agtgctgcag ctttaaggaa
1921 tctcatggca aataggcctg cgaagtacaa ggatgccaat attatgtctc ctggctcaag
1981 cttgccatct cttcatgtta ggaaacaaaa agccctagaa gcagaattag atgctcagca
2041 cttatcagaa acttttgaca atatagacaa tttaagtccc aaggcatctc atcgtagtaa
2101 gcagagacac aagcaaagtc tctatggtga ttatgttttt gacaccaatc gacatgatga
2161 taataggtca gacaatttta atactggcaa catgactgtc ctttcaccat atttgaatac
2221 tacagtgtta cccagctcct cttcatcaag aggaagctta gatagttctc gttctgaaaa
2281 agatagaagt ttggagagag aacgcggaat tggtctaggc aactaccatc cagcaacaga
2341 aaatccagga acttcttcaa agcgaggttt gcagatctcc accactgcag cccagattgc
2401 caaagtcatg gaagaagtgt cagccattca tacctctcag gaagacagaa gttctgggtc
2461 taccactgaa ttacattgtg tgacagatga gagaaatgca cttagaagaa gctctgctgc
2521 ccatacacat tcaaacactt acaatttcac taagtcggaa aattcaaata ggacatgttc
2581 tatgccttat gccaaattag aatacaagag atcttcaaat gatagtttaa atagtgtcag
2641 tagtagtgat ggttatggta aaagaggtca aatgaaaccc tcgattgaat cctattctga
2701 agatgatgaa agtaagtttt gcagttatgg tcaataccca gccgacctag cccataaaat
2761 acatagtgca aatcatatgg atgataatga tggagaacta gatacaccaa taaattatag
2821 tcttaaatat tcagatgagc agttgaactc tggaaggcaa agtccttcac agaatgaaag
2881 atgggcaaga cccaaacaca taatagaaga tgaaataaaa caaagtgagc aaagacaatc
2941 aaggaatcaa agtacaactt atcctgttta tactgagagc actgatgata aacacctcaa
3001 gttccaacca cattttggac agcaggaatg tgtttctcca tacaggtcac ggggagccaa
3061 tggttcagaa acaaatcgag tgggttctaa tcatggaatt aatcaaaatg taagccagtc
3121 tttgtgtcaa gaagatgact atgaagatga taagcctacc aattatagtg aacgttactc
3181 tgaagaagaa cagcatgaag aagaagagag accaacaaat tatagcataa aatataatga
3241 agagaaacgt catgtggatc agcctattga ttatagttta aaatatgcca cagatattcc
3301 ttcatcacag aaacagtcat tttcattctc aaagagttca tctggacaaa gcagtaaaac
3361 cgaacatatg tcttcaagca gtgagaatac gtccacacct tcatctaatg ccaagaggca
3421 gaatcagctc catccaagtt ctgcacagag tagaagtggt cagcctcaaa aggctgccac
3481 ttgcaaagtt tcttctatta accaagaaac aatacagact tattgtgtag aagatactcc
3541 aatatgtttt tcaagatgta gttcattatc atctttgtca tcagctgaag atgaaatagg
3601 atgtaatcag acgacacagg aagcagattc tgctaatacc ctgcaaatag cagaaataaa
3661 agaaaagatt ggaactaggt cagctgaaga tcctgtgagc gaagttccag cagtgtcaca
3721 gcaccctaga accaaatcca gcagactgca gggttctagt ttatcttcag aatcagccag
3781 gcacaaagct gttgaatttt cttcaggagc gaaatctccc tccaaaagtg gtgctcagac
3841 acccaaaagt ccacctgaac actatgttca ggagacccca ctcatgttta gcagatgtac
3901 ttctgtcagt tcacttgata gttttgagag tcgttcgatt gccagctccg ttcagagtga
3961 accatgcagt ggaatggtaa gtggcattat aagccccagt gatcttccag atagccctgg
4021 acaaaccatg ccaccaagca gaagtaaaac acctccacca cctcctcaaa cagctcaaac
4081 caagcgagaa gtacctaaaa ataaagcacc tactgctgaa aagagagaga gtggacctaa
4141 gcaagctgca gtaaatgctg cagttcagag ggtccaggtt cttccagatg ctgatacttt
4201 attacatttt gccacggaaa gtactccaga tggattttct tgttcatcca gcctgagtgc
4261 tctgagcctc gatgagccat ttatacagaa agatgtggaa ttaagaataa tgcctccagt
4321 tcaggaaaat gacaatggga atgaaacaga atcagagcag cctaaagaat caaatgaaaa
4381 ccaagagaaa gaggcagaaa aaactattga ttctgaaaag gacctattag atgattcaga
4441 tgatgatgat attgaaatac tagaagaatg tattatttct gccatgccaa caaagtcatc
4501 acgtaaagca aaaaagccag cccagactgc ttcaaaatta cctccacctg tggcaaggaa
4561 accaagtcag ctgcctgtgt acaaacttct accatcacaa aacaggttgc aaccccaaaa
4621 gcatgttagt tttacaccgg gggatgatat gccacgggtg tattgtgttg aagggacacc
4681 tataaacttt tccacagcta catctctaag tgatctaaca atcgaatccc ctccaaatga
4741 gttagctgct ggagaaggag ttagaggagg ggcacagtca ggtgaatttg aaaaacgaga
4801 taccattcct acagaaggca gaagtacaga tgaggctcaa ggaggaaaaa cctcatctgt
4861 aaccatacct gaattggatg acaataaagc agaggaaggt gatattcttg cagaatgcat
4921 taattctgct atgcccaaag ggaaaagtca caagcctttc cgtgtgaaaa agataatgga
4981 ccaggtccag caagcatctg cgtcttcttc tgcacccaac aaaaatcagt tagatggtaa
5041 gaaaaagaaa ccaacttcac cagtaaaacc tataccacaa aatactgaat ataggacacg
5101 tgtaagaaaa aatgcagact caaaaaataa tttaaatgct gagagagttt tctcagacaa
5161 caaagattca aagaaacaga atttgaaaaa taattccaag gtcttcaatg ataagctccc
5221 aaataatgaa gatagagtca gaggaagttt tgcttttgat tcacctcatc attacacgcc
5281 tattgaagga actccttact gtttttcacg aaatgattct ttgagttctc tagattttga
5341 tgatgatgat gttgaccttt ccagggaaaa ggctgaatta agaaaggcaa aagaaaataa
5401 ggaatcagag gctaaagtta ccagccacac agaactaacc tccaaccaac aatcagctaa
5461 taagacacaa gctattgcaa agcagccaat aaatcgaggt cagcctaaac ccatacttca
5521 gaaacaatcc acttttcccc agtcatccaa agacatacca gacagagggg cagcaactga
5581 tgaaaagtta cagaattttg ctattgaaaa tactccggtt tgcttttctc ataattcctc
5641 tctgagttct ctcagtgaca ttgaccaaga aaacaacaat aaagaaaatg aacctatcaa
5701 agagactgag ccccctgact cacagggaga accaagtaaa cctcaagcat caggctatgc
5761 tcctaaatca tttcatgttg aagatacccc agtttgtttc tcaagaaaca gttctctcag
5821 ttctcttagt attgactctg aagatgacct gttgcaggaa tgtataagct ccgcaatgcc
5881 aaaaaagaaa aagccttcaa gactcaaggg tgataatgaa aaacatagtc ccagaaatat
5941 gggtggcata ttaggtgaag atctgacact tgatttgaaa gatatacaga gaccagattc
6001 agaacatggt ctatcccctg attcagaaaa ttttgattgg aaagctattc aggaaggtgc
6061 aaattccata gtaagtagtt tacatcaagc tgctgctgct gcatgtttat ctagacaagc
6121 ttcgtctgat tcagattcca tcctttccct gaaatcagga atctctctgg gatcaccatt
6181 tcatcttaca cctgatcaag aagaaaaacc ctttacaagt aataaaggcc cacgaattct
6241 aaaaccaggg gagaaaagta cattggaaac taaaaagata gaatctgaaa gtaaaggaat
6301 caaaggagga aaaaaagttt ataaaagttt gattactgga aaagttcgat ctaattcaga
6361 aatttcaggc caaatgaaac agccccttca agcaaacatg ccttcaatct ctcgaggcag
6421 gacaatgatt catattccag gagttcgaaa tagctcctca agtacaagtc ctgtttctaa
6481 aaaaggccca ccccttaaga ctccagcctc caaaagccct agtgaaggtc aaacagccac
6541 cacttctcct agaggagcca agccatctgt gaaatcagaa ttaagccctg ttgccaggca
6601 gacatcccaa ataggtgggt caagtaaagc accttctaga tcaggatcta gagattcgac
6661 cccttcaaga cctgcccagc aaccattaag tagacctata cagtctcctg gccgaaactc
6721 aatttcccct ggtagaaatg gaataagtcc tcctaacaaa ttatctcaac ttccaaggac
6781 atcatcccct agtactgctt caactaagtc ctcaggttct ggaaaaatgt catatacatc
6841 tccaggtaga cagatgagcc aacagaacct taccaaacaa acaggtttat ccaagaatgc
6901 cagtagtatt ccaagaagtg agtctgcctc caaaggacta aatcagatga ataatggtaa
6961 tggagccaat aaaaaggtag aactttctag aatgtcttca actaaatcaa gtggaagtga
7021 atctgataga tcagaaagac ctgtattagt acgccagtca actttcatca aagaagctcc
7081 aagcccaacc ttaagaagaa aattggagga atctgcttca tttgaatctc tttctccatc
7141 atctagacca gcttctccca ctaggtccca ggcacaaact ccagttttaa gtccttccct
7201 tcctgatatg tctctatcca cacattcgtc tgttcaggct ggtggatggc gaaaactccc
7261 acctaatctc agtcccacta tagagtataa tgatggaaga ccagcaaagc gccatgatat
7321 tgcacggtct cattctgaaa gtccttctag acttccaatc aataggtcag gaacctggaa
7381 acgtgagcac agcaaacatt catcatccct tcctcgagta agcacttgga gaagaactgg
7441 aagttcatct tcaattcttt ctgcttcatc agaatccagt gaaaaagcaa aaagtgagga
7501 tgaaaaacat gtgaactcta tttcaggaac caaacaaagt aaagaaaacc aagtatccgc
7561 aaaaggaaca tggagaaaaa taaaagaaaa tgaattttct cccacaaata gtacttctca
7621 gaccgtttcc tcaggtgcta caaatggtgc tgaatcaaag actctaattt atcaaatggc
7681 acctgctgtt tctaaaacag aggatgtttg ggtgagaatt gaggactgtc ccattaacaa
7741 tcctagatct ggaagatctc ccacaggtaa tactcccccg gtgattgaca gtgtttcaga
7801 aaaggcaaat ccaaacatta aagattcaaa agataatcag gcaaaacaaa atgtgggtaa
7861 tggcagtgtt cccatgcgta ccgtgggttt ggaaaatcgc ctgaactcct ttattcaggt
7921 ggatgcccct gaccaaaaag gaactgagat aaaaccagga caaaataatc ctgtccctgt
7981 atcagagact aatgaaagtt ctatagtgga acgtacccca ttcagttcta gcagctcaag
8041 caaacacagt tcacctagtg ggactgttgc tgccagagtg actcctttta attacaaccc
8101 aagccctagg aaaagcagcg cagatagcac ttcagctcgg ccatctcaga tcccaactcc
8161 agtgaataac aacacaaaga agcgagattc caaaactgac agcacagaat ccagtggaac
8221 ccaaagtcct aagcgccatt ctgggtctta ccttgtgaca tctgtttaaa agagaggaag
8281 aatgaaacta agaaaattct atgttaatta caactgctat atagacattt tgtttcaaat
8341 gaaactttaa aagactgaaa aattttgtaa ataggtttga ttcttgttag agggtttttg
8401 ttctggaagc catatttgat agtatacttt gtcttcactg gtcttatttt gggaggcact
8461 cttgatggtt aggaaaaaaa tagtaaagcc aagtatgttt gtacagtatg ttttacatgt
8521 atttaaagta gcatcccatc ccaacttcct ttaattattg cttgtcttaa aataatgaac
8581 actacagata gaaaatatga tatattgctg ttatcaatca tttctagatt ataaactgac
8641 taaacttaca tcagggaaaa attggtattt atgcaaaaaa aaatgttttt gtccttgtga
8701 gtccatctaa catcataatt aatcatgtgg ctgtgaaatt cacagtaata tggttcccga
8761 tgaacaagtt tacccagcct gctttgcttt actgcatgaa tgaaactgat ggttcaattt
8821 cagaagtaat gattaacagt tatgtggtca catgatgtgc atagagatag ctacagtgta
8881 ataatttaca ctattttgtg ctccaaacaa aacaaaaatc tgtgtaactg taaaacattg
8941 aatgaaacta ttttacctga actagatttt atctgaaagt aggtagaatt tttgctatgc
9001 tgtaatttgt tgtatattct ggtatttgag gtgagatggc tgctctttta ttaatgagac
9061 atgaattgtg tctcaacaga aactaaatga acatttcaga ataaattatt gctgtatgta
9121 aactgttact gaaattggta tttgtttgaa gggtcttgtt tcacatttgt attaataatt
9181 gtttaaaatg cctcttttaa aagcttatat aaattttttt cttcagcttc tatgcattaa
9241 gagtaaaatt cctcttactg taataaaaac aattgaagaa gactgttgcc acttaaccat
9301 tccatgcgtt ggcacttatc tattcctgaa atttctttta tgtgattagc tcatcttgat
9361 ttttaatatt tttccactta aacttttttt tcttactcca ctggagctca gtaaaagtaa
9421 attcatgtaa tagcaatgca agcagcctag cacagactaa gcattgagca taataggccc
9481 acataatttc ctctttctta atattataga attctgtact tgaaattgat tcttagacat
9541 tgcagtctct tcgaggcttt acagtgtaaa ctgtcttgcc ccttcatctt cttgttgcaa
9601 ctgggtctga catgaacact ttttatcacc ctgtatgtta gggcaagatc tcagcagtga
9661 agtataatca gcactttgcc atgctcagaa aattcaaatc acatggaact ttagaggtag
9721 atttaatacg attaagatat tcagaagtat attttagaat ccctgcctgt taaggaaact
9781 ttatttgtgg taggtacagt tctggggtac atgttaagtg tccccttata cagtggaggg
9841 aagtcttcct tcctgaagga aaataaactg acacttatta actaagataa tttacttaat
9901 atatcttccc tgatttgttt taaaagatca gagggtgact gatgatacat gcatacatat
9961 ttgttgaata aatgaaaatt tatttttagt gataagattc atacactctg tatttgggga
 '''

### Instantiate your class and read in the DNA sequence

In [212]:
a = PrimerDesign("BIODW")
dna = dna_sequence
b = a.set_dna_sequence(dna)


### If you need to adjust any parameter from their default values in the init method, do it here

In [213]:
#all default values stay

### Show the outcome of your testing and the functions in the subsequent cells 

In [214]:
fp,rp,x=a.func_simulated_annealing()      
print(a.cost_objective_function_info(fp,rp))

===Forward Primer=== gtcctgttcctatgggttc
Criterion                       Cost function score
---------------------------------------------------
length                                        0.000
annealing temperature                         0.000
%cg_content                                   0.000
specificity                                   0.000
runs                                          0.000
repeats                                       0.000


===Reverse Primer=== gttgttcttccatcgcaact
Criterion                       Cost function score
---------------------------------------------------
length                                        0.000
annealing temperature                         0.000
%cg_content                                   0.000
specificity                                   0.000
runs                                          0.000
repeats                                       0.000

Total costs:                                  0.000

Temperature Difference       

In [215]:
QUESTIONS

• What were the simulation parameters that you used (in Step 1.1 of the
algorithm)?

I set the initial temperature to be the maximum annealing temperature, which is (0.6*22*4) + (0.4*22*2) = 70.4 degrees Celcius.
The stopping temperature is set to 0.01 so as to ensure that the method runs sufficient tests to generate the best primer pairs for the optimal cost objective function.
Decreasing factor is set as 0.999 to create the slightest increment in temperature so as to generate sufficient tests for optimal solution.

• From the output of your code, how well does the final solution meet all
the criteria?

The output of my code generates primer pairs that have 0 value for all the criteria cost functions, indicating the optimal cost objective function. 
Hence, they meet the criteria and are suitable for PCR.

• How many iterations were required to obtain this solution?
8854 iterations were required. ( number of iterations = (ln(0.01/70.4))/ln(0.999) )

• Will a brute-force algorithm (one that considers every single possible
solution) require fewer or more iterations? Explain.








SyntaxError: invalid character in identifier (<ipython-input-215-524a050aa62c>, line 3)